In [ ]:
import os
import sys
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc, recall_score
from sklearn.model_selection import train_test_split

# Define functions to read TSV files
def read_tsv(filename, inf_ind, skip_1st=False, file_encoding="utf8"):
    extract_inf = []
    with open(filename, "r", encoding=file_encoding) as tsv_f:
        if skip_1st:
            tsv_f.readline()
        line = tsv_f.readline()
        while line:
            line_list = line.strip().split("\t")
            temp_inf = [line_list[ind] for ind in inf_ind]
            extract_inf.append(temp_inf)
            line = tsv_f.readline()
    return extract_inf

# Define a function that reads an amino acid feature file and generates a feature dictionary
def get_features(filename, f_num=15):
    f_list = read_tsv(filename, list(range(16)), True)
    f_dict = {}
    left_num = 0
    right_num = 0
    if f_num > 15:
        left_num = (f_num - 15) // 2
        right_num = f_num - 15 - left_num
    for f in f_list:
        f_dict[f[0]] = [0] * left_num + [float(x) for x in f[1:]] + [0] * right_num
    f_dict["X"] = [0] * f_num
    return f_dict

# Defining Input Functions
def generate_input(sps, sp_lbs, feature_dict, feature_num, ins_num, max_len):
    xs, ys = [], []
    i = 0
    for sp in sps:
        xs.append([[[0] * feature_num] * max_len] * ins_num)
        ys.append(sp_lbs[i])
        j = 0
        for tcr in sp:
            tcr_seq = tcr[0]
            right_num = max_len - len(tcr_seq)
            tcr_seq += "X" * right_num
            tcr_matrix = []
            for aa in tcr_seq:
                tcr_matrix.append(feature_dict[aa.upper()])
            xs[i][j] = tcr_matrix
            j += 1
        i += 1
    xs = np.array(xs)
    xs = xs.swapaxes(2, 3)
    ys = np.array(ys)
    return xs, ys


#Define the Generate Label function
def load_data(sample_dir):
    training_data = []
    training_labels = []
    for sample_file in os.listdir(sample_dir):
        training_data.append(read_tsv(os.path.join(sample_dir, sample_file), [0, 1], True))
        if "P" in sample_file:
            training_labels.append(1)
        elif "H" in sample_file:
            training_labels.append(0)
        else:
            print("Wrong sample filename! Please name positive samples with 'P' and negative samples with 'H'.")
            sys.exit(1)
        
    return training_data, training_labels

#Define the evaluation function
def evaluate(model, criterion, test_loader, device="cuda"):
    test_total_loss = 0.0
    all_preds = []
    all_labels = []
    
    model.eval()
    with torch.no_grad():
        for test_batch_x, test_batch_y in test_loader:
            test_batch_x = test_batch_x.to(device)
            test_batch_y = test_batch_y.to(device).view(-1, 1)  
            test_pred = model(test_batch_x)

            test_loss = criterion(test_pred, test_batch_y)
            test_total_loss += test_loss.item()
            all_preds.append(test_pred.cpu().numpy())
            all_labels.append(test_batch_y.cpu().numpy())
            
        test_avg_loss = test_total_loss / len(test_loader)
        return test_avg_loss, all_preds, all_labels
    
     
#Define the training function    
from tqdm import tqdm
def train(model, criterion, optimizer, train_loader, valid_loader, epoches=100, device="cuda"):
    
    
    model_path = f'../Model Test Files/{Cancer_name}checkpoint.pth'  # Save path of the model file
    early_stopping = EarlyStopping(PATIENCE, path=model_path, verbose=False)
    

    epoch_train_losses = []
    epoch_test_losses = []
    with tqdm(total=epoches) as t:
        t.set_description(f'{Cancer_name}')
        for epoch in range(epoches):
            model.train()
            total_loss = 0.0
            for batch_x, batch_y in train_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device).view(-1, 1)  
                pred = model(batch_x)

                loss = criterion(pred, batch_y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            
           
            avg_loss = total_loss / len(train_loader)
            epoch_train_losses.append(avg_loss)
            test_avg_loss, _, _ = evaluate(model, criterion, test_loader, device)
            epoch_test_losses.append(test_avg_loss) 
            t.set_postfix(loss=avg_loss, test_loss=test_avg_loss)
            t.update(1)
            early_stopping(test_avg_loss, model)
            
            if early_stopping.early_stop:
                model.load_state_dict(torch.load(model_path))
                #print('Early stopping')
                break

                
#Define CNNLayer                
class CNNLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(CNNLayer, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.adaptive_maxpool = nn.AdaptiveMaxPool1d(1)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.adaptive_maxpool(x)
        return x
    
#Define PositionalEncoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.3, max_len=100):
        super(PositionalEncoding, self).__init__()
        
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
      
        pe = torch.zeros(max_len, 1, d_model)
        
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe)

    def forward(self, x):        
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
       
#Define AutoTFCNNY    
class AutoTFCNNY(nn.Module):
    def __init__(self, feature_num, hidden_size, output_size, num_heads, num_layers, dropout, max_len, ins_num):
        super(AutoTFCNNY, self).__init__()
        
        self.hidden_size = hidden_size  
        self.ins_num = ins_num

        self.input_embedding = nn.Linear(feature_num, hidden_size)
        
        self.pos_encoder = PositionalEncoding(hidden_size, dropout, max_len)

        
        self.cnn1 = CNNLayer(hidden_size, hidden_size, kernel_size=8, stride=1, padding=4)
        
        self.batchnorm1 = nn.BatchNorm1d(hidden_size)  

        self.encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layers, num_layers=num_layers)

        
        self.cnn2 = CNNLayer(hidden_size, hidden_size, kernel_size=8, stride=1, padding=4)
        
        self.batchnorm2 = nn.BatchNorm1d(hidden_size)  
        self.dropout = nn.Dropout(p=dropout)  

        self.decoder = nn.Linear(hidden_size, output_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.input_embedding.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        src = src.flatten(0, 1)
        src = src.permute(2, 0, 1)
        src = self.input_embedding(src)
        src = self.pos_encoder(src)

        
        src = src.permute(1, 2, 0) 
        src = self.cnn1(src)
        src = self.batchnorm1(src)  
        src = src.permute(2, 0, 1)  

        output = self.transformer_encoder(src)

        
        output = output.permute(1, 2, 0)
        output = self.cnn2(output)
        output = self.batchnorm2(output)  
        output = output.permute(2, 0, 1)

        output = output[0]
        output = output.view(-1, self.ins_num, self.hidden_size)

        
        output = self.dropout(output)

        output = output.mean(dim=1)
        output = self.decoder(output)

        return output

# Setting model parameters     
def init_model():
    
    feature_num = 15  
    hidden_size = 30  
    output_size = 1  
    num_heads = 6  
    num_layers = 2  
    dropout = 0.6  
    max_len = 24  
    ins_num = 100  

    model = AutoTFCNNY(feature_num, hidden_size, output_size, num_heads, num_layers, dropout, max_len, ins_num)
    return model
    

In [ ]:
# Introduce an early stop mechanism
sys.path.append('../')
from python_codes.pytorchtools import EarlyStopping

# Read the amino acid characterization file
aa_file = "../Data/PCA15.txt"
aa_vectors = get_features(aa_file)  

BATCH_SIZE = 64   # Batch size
NUM_EPOCHES = 2000 #Total number of training rounds
PATIENCE = 300    # Set the patience value for early stops

all_accuracies = []  
all_sensitivities = [] 
all_specificities = [] 
all_aucs = []  

device = "cuda"

Cancer_list = ["UBC","BRCA","NSCLC","Melanoma","CRC"] # Cancer
results = []
results_ROC = []

for Cancer_name in Cancer_list:
    data_dir = f'../Data/{Cancer_name}'   # Cancer File Path
    training_data, training_labels = load_data(data_dir)
    print(f"Working on {Cancer_name} dataset: {len(training_data)} samples")
    
    
    train_data, test_data, train_labels, test_labels = train_test_split(training_data, training_labels, test_size=0.2, random_state=1234)
   
    train_input_batch, train_label_batch = generate_input(train_data, train_labels, aa_vectors, 15, 100, 24)
        
    train_input_batch, train_label_batch = torch.Tensor(train_input_batch).to(torch.device("cuda")), torch.LongTensor(train_label_batch).to(torch.device("cuda"))

    
    test_input_batch, test_label_batch = generate_input(test_data, test_labels, aa_vectors, 15, 100, 24)
    test_input_batch, test_label_batch = torch.Tensor(test_input_batch).to(torch.device("cuda")), torch.LongTensor(test_label_batch).to(torch.device("cuda"))    

     
    
    train_label_batch = train_label_batch.float()

    test_label_batch = test_label_batch.float()        
        

    train_dataset = Data.TensorDataset(train_input_batch, train_label_batch)
    test_dataset = Data.TensorDataset(test_input_batch, test_label_batch)

    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


    model = init_model().to(device)
    criterion = nn.BCEWithLogitsLoss() 
    optimizer = optim.Adam(model.parameters(), lr=0.0005)

    train(model, criterion, optimizer, train_loader, test_loader, epoches=NUM_EPOCHES, device=device)
